## Оценка качества STT моделей

Качество распознавания речи на имеющихся данных сильно зависит от выбора конкретной модели. Но чтобы можно было определить, какая из моделей лучше справляется с распознаванием под конкретный бизнес-кейс, требуется правильно построить систему оценки качества распознавания и определить соответствующие метрики.

Наиболее популярной метрикой при оценке качества распознавания является метрика WER (Word Error Rate). Эта метрика оценивает похожесть полученного распознавания на некоторый "эталонный" пример, как правило получаемый с помощью разметки аудиозаписей с помощью асессоров. Проблема заключается в том, что значение этой метрики может очень сильно варьироваться не только от результатов распознавания, но также и от качества разметки аудиозаписей и самой методологии оценки.

В качестве примера можно привести популярную фразу "алло", которая может быть размечена как минимум четырьмя различными способами: "алло", "алле", "ало", "але". Также качество может падать, если пытаться различать буквы "ё" и "е", которые эквивалентны с точки зрения большинства моделей распознавания. Наконец, качество может очень сильно зависеть от того, требуется ли нам различать различные формы одних и тех же слов (к примеру, род, падежи существительных, времена глаголов и т.п.).

По этой причине мы решили предоставить пользователям свою небольшую библиотеку, которая позволит использовать метрики для оценки качества распознавания с учётом описанных особенностей. На текущий момент эта библиотека поддерживает вычисление метрики WER, однако в дальнейшем библиотеку планируется расширить и другими метриками.

In [11]:
from stt_metrics import WER, ClusterReferences
from stt_metrics.text_transform import Lemmatizer

## Пример использования метрики WER

Рассмотрим самый простой вариант использования метрики WER:

In [12]:
reference = 'алло добрый день с моей карты только что списали крупную сумму денег хочу заблокировать её'
hypothesis = 'але добрый день моей карты только что списал крупную суму денег хочу заблокировать ее'

In [13]:
wer = WER()
wer_data = wer.get_metric_data(hyp=hypothesis, ref=reference)
wer_value = wer.calculate_metric(wer_data)

`wer_data` &mdash; специальный объект, который хранит необходимую для вычисления WER информацию, а также предоставляет нам выравнивание двух текстов с указанием отличий. Последняя особенность может быть очень полезна при дальнейшем анализе отличий в распознавании и разметке.

Так, на примере ниже мы видим, что значение метрики WER оказывается достаточно высоким. Однако многие ошибки для данного кейса оказываются довольно незначительными. Так, одна ошибка возникает из-за появившейся в разметке буквы "ё", ещё одна из ошибок связана с отличием в словах "але" и "алло", и ещё одна ошибка произошла из-за нераспознанного множественного числа глагола "списал".

In [14]:
print(wer_data)
print(f'WER: {wer_value}')

{
  errors: 5,
  hyp_wc: 14,
  ref_wc: 15,
  diff_hyp: АЛЕ  добрый день * моей карты только что СПИСАЛ  крупную СУМУ  денег хочу заблокировать ЕЕ,
  diff_ref: АЛЛО добрый день с моей карты только что СПИСАЛИ крупную СУММУ денег хочу заблокировать ЕЁ
}
WER: 0.3333333333333333


### Избавление от ошибок

#### Удаление артефактов

От возможных ошибок первого типа довольно просто избавиться, если заранее провалидировать имеющуюся разметку аудио и избавиться от возможных артефактов. В данном случае такой препроцессинг оказывается довольно простым, хотя в общем случае артефакты могут быть и куда менее очевидными.

In [15]:
def remove_artifacts(text):
    return text.replace('ё', 'е')


hypothesis, reference = remove_artifacts(hypothesis), remove_artifacts(reference)

wer = WER()
wer_data = wer.get_metric_data(hyp=hypothesis, ref=reference)
wer_value = wer.calculate_metric(wer_data)

print(wer_data)
print(f'WER: {wer_value}')

{
  errors: 4,
  hyp_wc: 14,
  ref_wc: 15,
  diff_hyp: АЛЕ  добрый день * моей карты только что СПИСАЛ  крупную СУМУ  денег хочу заблокировать ее,
  diff_ref: АЛЛО добрый день с моей карты только что СПИСАЛИ крупную СУММУ денег хочу заблокировать ее
}
WER: 0.26666666666666666


#### ClusterReferences

Ошибки второго типа также могут быть довольно просто исключены, если явно указать наборы синонимичных фраз (ClusterReferences), которые следует воспринимать одинаково:

In [16]:
cr = ClusterReferences()
cr.add_cluster(center='алло', aliases=['ало', 'але', 'алле'])

wer = WER(cr=cr)
wer_data = wer.get_metric_data(hyp=hypothesis, ref=reference)
wer_value = wer.calculate_metric(wer_data)

print(wer_data)
print(f'WER: {wer_value}')

{
  errors: 3,
  hyp_wc: 14,
  ref_wc: 15,
  diff_hyp: алло добрый день * моей карты только что СПИСАЛ  крупную СУМУ  денег хочу заблокировать ее,
  diff_ref: алло добрый день с моей карты только что СПИСАЛИ крупную СУММУ денег хочу заблокировать ее
}
WER: 0.2


#### Lemmatizer

Наконец, можно также постараться избавиться и от ошибок третьего типа, если привести все имеющиеся слова к их леммам. Тем не менее, делать это также следует аккуратно, потому что в лемматизации также могут нуждаться и фразы из ClusterReferences:

In [17]:
lemmatizer = Lemmatizer()
lemm_hypothesis, lemm_reference = lemmatizer.transform(hypothesis), lemmatizer.transform(reference)

wer = WER(cr=cr)
wer_data = wer.get_metric_data(hyp=lemm_hypothesis, ref=lemm_reference)
wer_value = wer.calculate_metric(wer_data)

print(wer_data)
print(f'WER: {wer_value}')

{
  errors: 2,
  hyp_wc: 14,
  ref_wc: 15,
  diff_hyp: алло добрый день * мой карта только что списать крупный СУМА  деньга хотеть заблокировать она,
  diff_ref: алло добрый день с мой карта только что списать крупный СУММА деньга хотеть заблокировать она
}
WER: 0.13333333333333333


### Выводы

Если вам просто требуется проверить, что конкретная модель распознавания в среднем делает достаточно мало ошибок в словах, то простая версия метрики WER вполне может подойти вам.

Если же вы используете эту метрику для сравнения поведения различных моделей на вашем бизнес-кейсе, то вам может также помочь:

* Удаление из разметки и набор распознаваний явных артефактов, которые могут ухудшать значения используемых метрик, не влияя при этом на качество решения конкретной бизнес-задачи

* Использование ClusterReference'ов для "склейки" одинаковых по сути распознаваний. Так можно склеить различные варианты распознавания фраз типа "алло", а также распознавания фраз с пробелами (наподобие "контр страйк" и "контрстрайк")

* Лемматизация слов, если для нас не важны их окончания

## Пример использования метрики WER (агрегированный)

Приведём ещё пример того, как просто посчитать значение метрики для нескольких записей.

Воспользуемся для этого функцией `evaluate_wer`, которая считает WER сразу для большого набора текстов. 

Заметим, что поскольку обычно в случае STT и гипотезы, и тексты разметки привязаны к конкрентым записям, которые в свою очередь имеют уникальные идентификаторы (в виде того же названия), то эта функция принимает на вход список текстов в виде словаря, где ключом является некоторый уникальный идентификатор, а значением &mdash; сам текст гипотезы/разметки.

Выходом же этой функции является пара значений. Первое значение &mdash; среднее значение WER по всем переданным парам текстов. Второе значение &mdash; словарь, который для каждой пары текстов хранит соответствующий им WER, а также выравнивание текстов с указанием отличий.

In [18]:
from stt_metrics import evaluate_wer


mean_wer, full_stats = evaluate_wer(
    references={'wav1': 'один два', 'wav2': 'один два'},
    hypotheses={'wav1': 'один три', 'wav2': 'один один три'},
    cluster_references=None
)

In [19]:
mean_wer

0.5833333333333333

In [10]:
full_stats

{
  "id1": {
    "metric_value": 0.5,
    "metric_data": {
      "errors": 1,
      "hyp_wc": 2,
      "ref_wc": 2,
      "diff_hyp": "один ТРИ",
      "diff_ref": "один ДВА"
    },
    "reference": "один два",
    "hypothesis": "один три"
  },
  "id2": {
    "metric_value": 0.6666666666666666,
    "metric_data": {
      "errors": 2,
      "hyp_wc": 3,
      "ref_wc": 2,
      "diff_hyp": "один ОДИН три",
      "diff_ref": "один ДВА  ***"
    },
    "reference": "один два",
    "hypothesis": "один один три"
  }
}
